# Analysis of Wage and Hour Investigations

This notebook analyzes investigations concluded by the Department of Labor's Wage and Hour Division, based on the division's WHISARD database, which BuzzFeed News obtained via a Freedom of Information Act request. The database contains information on employers, violations, fines, and other details corresponding to investigations concluded between October 1, 2001 and March 31, 2015. For more details, see [the main project page](https://github.com/BuzzFeedNews/2016-05-H-2-debarments-and-violations).

## Load the data

In [1]:
import pandas as pd
import whisard_loader

*Note: `whisard_loader` contains custom code to simplify the process of loading data from the text files WHD provided. It is available [here](https://github.com/BuzzFeedNews/2016-05-H-2-debarments-and-violations/blob/master/notebooks/whisard_loader.py).*

In [2]:
employers = whisard_loader.load_employers().set_index("CASE_ID")

In [3]:
cases = whisard_loader.load_cases().set_index("CASE_ID")

In [4]:
violations = whisard_loader.load_violations().set_index("CASE_ID")

In [5]:
act_summaries = whisard_loader.load_act_summaries()

## Filter to just cases with H-2 violations and concluded 2010–2014

In [6]:
h2_act_summaries = act_summaries[
    act_summaries["ACT_ID"].isin([ "H2A", "H2B" ]) &
    (act_summaries["TTL_ACT_VIOLATIONS"] > 0)
]
h2_case_ids = h2_act_summaries["CASE_ID"]

In [7]:
h2_cases_2010_2014 = cases[
    cases.index.isin(h2_case_ids) &
    (cases["DATE_CONCLUDED_YEAR"] >= 2010) &
    (cases["DATE_CONCLUDED_YEAR"] <= 2014)
]

## Count total cases by year and type

In [8]:
tools_by_frequency = h2_cases_2010_2014["INVEST_TOOL_DESC"].value_counts().index.tolist()
tools_by_frequency

['Full Investigation',
 'Limited Investigation',
 'Conciliation',
 'Self Audit',
 'Office Audit']

In [9]:
h2_case_counts = h2_cases_2010_2014.reset_index().groupby([
    "DATE_CONCLUDED_YEAR",
    "INVEST_TOOL_DESC"
])["CASE_ID"].nunique()\
    .unstack()\
    .fillna(0)\
    [tools_by_frequency]
h2_case_counts["[total]"] = h2_case_counts.sum(axis=1)

### Number of cases w/H-2 violations, by overall investigation type and year concluded:

In [10]:
h2_case_counts

INVEST_TOOL_DESC,Full Investigation,Limited Investigation,Conciliation,Self Audit,Office Audit,[total]
DATE_CONCLUDED_YEAR,,,,,,
2010,92,16,0,0,0,108
2011,198,17,1,0,1,217
2012,240,30,1,1,0,272
2013,228,21,0,0,0,249
2014,202,34,0,0,0,236


In [11]:
h2_case_counts.sum()

INVEST_TOOL_DESC
Full Investigation        960
Limited Investigation     118
Conciliation                2
Self Audit                  1
Office Audit                1
[total]                  1082
dtype: float64

### Distinct employer IDs associated with the Full/Limited investigations above:

In [12]:
h2_cases_2010_2014[
    h2_cases_2010_2014["INVEST_TOOL_DESC"].isin([ 
        "Full Investigation", "Limited Investigation" 
    ])
].join(employers)["employer_id"].nunique()

995

## Altendorf Transport investigations

*Note: Altendorf Harvesting is a DBA of Altendorf Transport.*

In [13]:
altendorf_cases = employers[
    employers["ER_TRADE_NAME"].str.upper().str.contains(r"ALTENDORF ", na=False) |
    employers["ER_LEGAL_NAME"].str.upper().str.contains(r"ALTENDORF ", na=False)
][[
    "ER_LEGAL_NAME", 
    "ER_TRADE_NAME", 
    "ER_CITY", 
    "ER_STATE_ID" 
]].join(cases[cases["TTL_CASE_VIOLATION"] > 0][[
    "INVEST_TOOL_DESC",
    "DATE_CONCLUDED_YEAR",
    "TTL_CASE_VIOLATION",
    "AMT_BW_ASSESSED"
]], how="inner").sort_values("DATE_CONCLUDED_YEAR")

In [14]:
altendorf_cases.T

CASE_ID,1410202,1504154,1589410,1589471
ER_LEGAL_NAME,"Altendorf Transport, Inc.","Altendorf Transport, Inc.",Altendorf Harvesting,Altendorf Transport Inc.
ER_TRADE_NAME,Altendorf Harvesting,Altendorf Harvesting,Altendorf Harvesting,Altendorf Harvesting
ER_CITY,Minto,Minto,Minto,Minto
ER_STATE_ID,ND,ND,ND,ND
INVEST_TOOL_DESC,Full Investigation,Conciliation,Conciliation,Full Investigation
DATE_CONCLUDED_YEAR,2006,2008,2010,2011
TTL_CASE_VIOLATION,56,1,1,10
AMT_BW_ASSESSED,1956.12,1214.5,690.03,19158.9


In [15]:
altendorf_cases["AMT_BW_ASSESSED"].sum()

23019.510000000002

## T. Bell Detasseling investigations

In [16]:
t_bell_cases = employers[
    employers["ER_TRADE_NAME"].str.upper().str.contains(r"\bT.* BELL DETAS", na=False) |
    employers["ER_LEGAL_NAME"].str.upper().str.contains(r"\bT.* BELL DETAS", na=False)
][[
    "ER_LEGAL_NAME", 
    "ER_TRADE_NAME", 
    "ER_CITY", 
    "ER_STATE_ID" 
]].join(cases[[
    "INVEST_TOOL_DESC",
    "DATE_CONCLUDED_YEAR",
    "TTL_CASE_VIOLATION",
    "AMT_BW_ASSESSED"
]]).sort_values("DATE_CONCLUDED_YEAR")

In [17]:
t_bell_cases.T

CASE_ID,1622241,1704521,1732562
ER_LEGAL_NAME,T Bell Detasseling LLC,T. Bell Detasseling L.L.C.,"T Bell Detasseling, LLC"
ER_TRADE_NAME,T Bell Detasseling LLC,T. Bell Detasseling L.L.C.,"T Bell Detasseling, LLC"
ER_CITY,Lone Tree,Lone Tree,Lone Tree
ER_STATE_ID,IA,IA,IA
INVEST_TOOL_DESC,Full Investigation,Full Investigation,Full Investigation
DATE_CONCLUDED_YEAR,2012,2013,2015
TTL_CASE_VIOLATION,1413,1,52
AMT_BW_ASSESSED,0,0,0


In [18]:
t_bell_violations = violations[
    (violations["violation_found"] == True)
].ix[t_bell_cases.index]

In [19]:
t_bell_violations.groupby([ "ACT_ID", "VIOLATION_DESC"])\
    ["NUM_EES_REPRESENTD"].sum()\
    .sort_values(ascending=False)

ACT_ID  VIOLATION_DESC                                                  
H2A     23 Failed to make available required records (2008 & 2010 Rules)    1297
        30 FTC - abandonment/term for cause reqs (2008 & 2010 Rules)          58
        19 Failed to comply - 3/4-guarantee req (2008 & 2010 Rules)           58
        01 H-2A workers given preferential treatment (2008 & 2010 Rules)      50
MSPA    09 Fail to pay wages when due                                          1
        01 Fail to disclose conditions to workers                              1
H2A     48 H2ALC provided invalid "fixed-site" info (2008 & 2010 Rules)        1
Name: NUM_EES_REPRESENTD, dtype: float64

---

---

---